### Import Libary

In [14]:
import torch
import os
import yaml
import pandas as pd
import re

### Đọc danh sách file dữ liệu

### Đọc dữ liệu câu hỏi và câu trả lời, lưu thành 2 list

In [15]:

# Đọc file Excel
file_path = "data/original_data.xlsx"  # Thay bằng đường dẫn file Excel của bạn
df = pd.read_excel(file_path, usecols=["question", "answers"])

# Chuyển cột "question" thành danh sách
questions = df["question"].tolist()

# Chuyển cột "answer" thành danh sách
answers = df["answers"].tolist()

# In kết quả
print("Questions:", questions)
print("Answers:", answers)


Questions: ['Tên cơ sở đào tạo của trường là gì?', 'Mã trường của Trường Đại học Bách khoa - Đại học Đà Nẵng là gì?', 'Địa chỉ của trường là gì?', 'Trang thông tin điện tử của Trường Đại học Bách khoa là gì?', 'Trang thông tin tuyển sinh của trường là gì?', 'Trường có trang Facebook không? Nếu địa chỉ là gì?', 'Số điện thoại liên hệ tuyển sinh của trường là gì?', 'Thí sinh cần đáp ứng điều kiện gì để tham gia tuyển sinh?', 'Phạm vi tuyển sinh của Trường Đại học Bách khoa - Đại học Đà Nẵng là gì?', 'Trường Đại học Bách khoa - Đại học Đà Nẵng thuộc đơn vị nào?', 'Trường có bao nhiêu số điện thoại hỗ trợ tuyển sinh?', 'Trường Đại học Bách khoa - Đà Nẵng nằm ở quận nào của thành phố Đà Nẵng?', 'Thí sinh ở tỉnh khác có thể đăng ký xét tuyển vào trường không?', 'Trang tuyển sinh của trường có địa chỉ chính xác là gì?', 'Tên viết tắt của trường là gì?', 'Trường Đại học Bách khoa - Đại học Đà Nẵng có hệ đào tạo nào?', 'Làm thế nào để liên hệ với bộ phận tuyển sinh của trường?', 'Có thể cập nhậ

### Biến đổi các từ thành các cụm từ có nghĩa 

In [16]:
from pyvi import ViTokenizer

#! Đọc các cụm từ cần gộp từ file
def load_phrases(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        phrases = [line.strip() for line in file.readlines()]
    return phrases

file_path = 'phrases.txt'
custom_phrases = load_phrases(file_path)

def custom_tokenize(text):
    for phrase in custom_phrases:
        #! Gom nhóm các cụm từ có nghĩa
        text = text.replace(phrase, phrase.replace(" ", "_"))
    return ViTokenizer.tokenize(text)


def tokenize_sentences(sentences):
    result = []
    for sentence in sentences:
        result.append(custom_tokenize(sentence))
    return result


### Loại bỏ các ký tự đặc biệt và biến đổi thành chữ thường

In [17]:
def clean_text(text):
    # Loại bỏ khoảng trống dư thừa giữa các từ
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', text)
    text = re.sub(r'\!', '', text) # xoa dau cham than   
    text = re.sub(r'\?', '', text) # xoa dau hoi cham
    text = re.sub(r'\.', '', text) # xoa dau cham cau
    text = re.sub(r'#([^\s]+)', r'\1',text) # xóa các hastag #
    text =  re.sub(r'["\'()[]{}]', '', text) # xoa cac dau ngoac kep vs ngoac don
    text = text.lower()
    return text.strip()

df = pd.DataFrame({
    'questions': questions,
    'answers': answers,
})

df['questions_processed'] = df['questions'].apply(clean_text)
df['questions_processed'] = tokenize_sentences(df['questions_processed'])

df.to_csv('data.csv',index=False)
print(df.shape)


(370, 3)


### Load model và Tokenizer của phoBert

In [18]:
from transformers import AutoModel, AutoTokenizer

PhobertTokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
model = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")


### Embedding

In [19]:
sentences = df['questions_processed'].tolist()
inputs = PhobertTokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output

print(embeddings)
print(embeddings.shape)


tensor([[ 0.0387,  0.1673,  0.1168,  ..., -0.0950, -0.1295,  0.2398],
        [ 0.2258, -0.1711,  0.2621,  ..., -0.0998, -0.3288,  0.1852],
        [ 0.2428,  0.1397,  0.0877,  ..., -0.0598, -0.1974,  0.2529],
        ...,
        [-0.1775, -0.0111,  0.0241,  ...,  0.1944,  0.0706, -0.1551],
        [-0.0202,  0.2297,  0.0782,  ..., -0.0432, -0.0070,  0.1140],
        [-0.0223,  0.0632,  0.1447,  ..., -0.1203, -0.1178, -0.1010]])
torch.Size([370, 768])


In [20]:
embedding_list = embeddings.tolist()
df['embedding'] = embedding_list



In [21]:
import numpy as np

# Giả sử tensor_embeddings là tensor cần lưu
tensor_embeddings = embeddings.numpy()  # Chuyển từ Torch tensor sang NumPy array


np.save("embeddings.npy", tensor_embeddings)